## Skript zum Cleanen, Splitten und Parsen der Cases

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from utils import preprocessing
import re
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
import spacy
import datetime as dt
import math

### Einlesen der Cases

Die Datensatz wird in ein Pandas Dataframe eingelesen

In [ ]:
cases_df = pd.read_json('../2019-02-19_oldp_cases.json', lines=True)

Der Pandas Dataframe wird in eine Python Liste umgeformt und anschließend gelöscht, um Arbeitsspeicher freizugeben

In [ ]:
cases = []
for i in range(len(cases_df)) : 
    cases.append(cases_df.loc[i].to_dict())
    
del cases_df

### Cleanen der eingelesenen Cases 

Methode: clean

IN: Der zu cleanende Content

OUT: Der gecleante Content

In dieser Funktion werden unter anderem die HTML-Tags sowie Whitespaces und XML-Entity-Schreibweisen entfernt. Zudem werden zwischen Aufzählungen und dem nachfolgenden Text jeweils ein Punkt sowie ein Leerzeichen eingefügt, um die Aufzählung vom Text zu separieren. Ist bereits ein Punkt nach der Aufzählung vorhanden, so wird nur ein Leerzeichen ergänzt. Selbes gilt für Aufzählungen gefolgt von Anführungszeichen.
Potentielle Tippfehler, wie etwa ",." werden durch "." ersetzt, ":." durch ":".

"\n" werden bewusst weiterhin im Text behalten und anschließend in der Methode "get_case_dict" benutzt um zusätzliche Satztrennungen durchzuführen.

In [ ]:
def clean(content):
    soup = BeautifulSoup(content)
    content = soup.get_text()
    content = preprocessing.remove_whitespace(content)
    
    content = re.sub(r'(\d)([a-zA-ZäöüÄÖÜß])', "\g<1>. \g<2>", content)
    content = re.sub(r'(\d)(\.)([a-zA-ZäöüÄÖÜß])', "\g<1>\g<2> \g<3>", content)
    content = re.sub(r'(\d)(\„)([a-zA-ZäöüÄÖÜß])', "\g<1>. \g<2>\g<3>", content)
    
    content = content.replace(",.",".")
    content = content.replace(":.",":")
      
    return content

Ausführen der clean-Methode auf den Content jedes einzelnen Cases und Zurückschreiben des gecleanten Inhaltes in den Case selbst.

In [ ]:
i = 0
for case in cases:
    cleaned_content = clean(case['content'])
    case['content']=cleaned_content
    i = i+1
    
    if(i % 10000 == 0):
        print(i)

Umformen von "cases" in einen Pandas Dataframe zur einfacheren Abspeicherung als csv-Datei

In [ ]:
cleaned_cases_df = pd.DataFrame(cases)

Angabe eines Pfades zur Ablage der csv-Datei

In [ ]:
export_file_path_csv = "../cleaned_cases.csv"

Abspeichern des Pandas Dataframes in eine csv-Datei und anschließendes Löschen des nicht mehr benötigten Dataframes

In [ ]:
cleaned_cases_df.to_csv(export_file_path_csv, header=True, sep='\t', encoding="utf-8-sig", index=False)
del cleaned_cases_df

### Vorbereitung zum Trainieren und späteren Ausführen des PunktSentenceTokenizers

Methode: prepare_for_split

IN: Der vorzubereitende Content

OUT: Der vorbereitete Content

In dieser Methode werden innerhalb von runden Klammern (...) Punkte durch "_" ersetzt, um zu verhindern, dass der PunktSentenceTokenizer innerhalb besagter Klammern Satzgrenzen erkennt. Dies ist besonders bei Klammern der Form "( Art.14, Abs.3)" etc. wichtig.

In [ ]:
def prepare_for_split(content):   
    regEx = r"\((.*?)\)"   # Regulärer Ausdruck um "( )" zu erkennen
    matches = (re.finditer(regEx, content))   #liefert alle Übereinstimmungen des regulären Ausdrucks im Text "content" zurück
    for m in matches:   #Ersetzen von "." durch "_" in den gefundenen Sätzen
        matchedText = (m.group())
        matchedText = matchedText.replace(".", "_")
        content = content[:m.span()[0]] + matchedText + content[m.span()[1]:]
    
    return content

Erstellen eines Trainings-Textes für den PunktSentenceTokenizer auf Basis der gecleanten und vorbereiteten Texte der einzelnen Cases. Hier wird auf die ersten 1000 Cases und deren Content zurückgegriffen, da ein Training über mehr Cases eine Verschlechterung der Performance des PunktSentenceTokenizers zur Folge hatte.

In [ ]:
text_train = ''
j = 0
for case in cases[:1000]:
    text_train += prepare_for_split(case['content'])
    j+=1
    if(j % 100 == 0):
        print(j)

### Trainieren des PunktSentenceTokenizer

Im Folgenden wird ein "PunktSentenceTokenizer" angelegt und Parametrisiert, mit Hilfe dessen im Späteren die Sätze gesplittet werden.

In [ ]:
trainer = PunktTrainer()   #Anlegen eines PunktTrainers
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text_train)   #Trainieren des PunktTrainers auf den vorbereiteten Trainings-Text
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())   #Erlernte Parameter an den "PunktSentenceTokenizer" übergeben

# Anzahl der bereits bekannten Abkürzungen
print(len(tokenizer._params.abbrev_types))

print("----------------------------------------")

#Definieren zusätzlicher Abkürzungen
abbrev = ["Abb", "Abk", "allg", "eigtl", "gegr", "Nrn", "jmd", "ugs", "urspr", "usw", "Aufl", "rn", "sog", "ggfs", "insg", \
        "Abs", "GVG", "nr", "Art", "f", "ff", '"', "-", "vgl", "bzw", "bspw", "etc", "rn", "strspr", "bverwg", "Anm", \
        "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", \
        "s", "t", "u", "v", "w", "x", "y", "z"]

for abb in abbrev:
    tokenizer._params.abbrev_types.add(abb.lower())   #Hinzufügen der zusätzlichen Abkürzungen zum "PunktSentenceTokenizer"
    

fobj = open("german_abbreviations.txt")  #Hinzufügen von 4262 deutschen Abkürzungen zum "PunktSentenceTokenizer"
for line in fobj:
    line = line.lower()
    if(line[-1] == "."):
        line = line[:-1] 
    tokenizer._params.abbrev_types.add(line)
fobj.close()

print(len(tokenizer._params.abbrev_types))   # Anzahl der nun bekannten Abkürzungen

### Splitten der vorhandene Cases, parsen und abspeichern in ein Dictionary

Um einen erheblichen Geschwindigkeitsgewinn zu erhalten bietet es sich an den Dependence-Parser (spacy) auf der GPU auszuführen. Dies wird mit nachfolgendem Befehl erreicht

In [ ]:
isUSingGPU = spacy.require_gpu()

Kontrolle, ob die GPU tatsächlich verwendet wird

In [ ]:
isUSingGPU

Laden von "de_core_news_lg" in spacy

In [ ]:
nlp = spacy.load('de_core_news_lg')
nlp.max_length = 150000000

Methode: get_case_dict

IN: Der Bereich innerhalb der Liste "cases", für welchen das case_dict angelegt werden soll.

OUT: Das erzeugte case_dict für den vorgegebene Bereich der Liste "cases"

In dieser Funktion wird für den jeweiligen Bereich der cases-Liste ein Dictionary angelegt, welches die einzelnen Cases nach Case-ID als Key unterscheidet und als Value ein weiteres Dictionary enthält. Das zweite Dictioanry besitzt als Key die jeweilige Satznummer und als Value ein Tuple. Dieses Tuple wiederum besteht aus dem Satz als Volltext und einem weiteren Dictionary, welches die Zuordnung der einzelnen Wörter des Satzes zu "ROOT", "Subjekt", "AkkusativObjekt", "DativObjekt", "Rest" übernimmt.

Somit steht nach Ausführung dieser Funktion für die entsprechenden Cases ein Dictionary zur Verfügung, welches jeden Case anhand der Case-ID identifiziert, den Content des Cases in Sätze aufschlüsselt und die entsprechenden Sätze als Volltext, aber auch die Zuordnung der einzelnen Wörter zu oben beschriebenen Schlüsseln, enthält.

Entscheidente Geschwindigkeitserhöhung bringt neben der bereits zuvor beschriebenen Verwendung der GPU für Spacy auch die Anwendung einer von Spacy zur Verfügung gestellten Pipeline, welche es ermöglicht, nicht nur mit einzelnen Sätzen, sondern mit Batches von Sätzen zu arbeiten. Mehr dazu in der schriftlichen Ausarbeitung. 

In [ ]:
def get_case_dict(case_range):
    case_dict={}   #Dictionary welches die Cases anhand der Case-ID unterscheidet und ein Dictionary für Sätze enthält
    print("started at: ", str(dt.datetime.now()))   #Ausgabe um die Verarbeitunszeit zu tracken
    counter = 1   #counter für die Anzahl der bereits verarbeiteten Cases
    isUsingGPU = spacy.require_gpu()   #Abfrage um nachvollziehen zu können ob die GPU verwendet wird
    print("GPU? ", isUsingGPU)   #Ausgabe ob die GPU verwendet wird
    for case in cases[case_range[0]:case_range[1]]:   #Durchlaufen aller Cases im angegeben Bereich
        x=0   #Laufvariable für die Satznummer
        temp = tokenizer.tokenize(prepare_for_split(case['content'])) #Vorbereiten des Contents und anschließendes Splitten mit dem PunktSentenceTokenizer 
        split_sentences = []   #Liste für die gesplitteten Sätze
        for s in temp:   #Durchlaufen des Ergebnisses des PunktSentenceTokenizers für die Nachbearbeitung der Ergebnisse
            split_sentences += s.split("\n")   #Splitten anhand "\n"
            
        #Die in "prepare_for_split" hinzugefügten "_" wieder durch "." ersetzen und unnötige Leerzeichen an Satz-Anfang/ - Ende entfernen
        split_sentences = [s.strip().replace("_", ".") for s in split_sentences]   
        case_docs = list(nlp.pipe(split_sentences))   #Parsen der Sätze und ablegen der Ergebnisse in case_docs

        sentence_dict={}   #Dictionary, welches als Key die Satznummer hat und als Value ein Tuple aus Volltext und Wortzuordnungen
        for case_doc in case_docs:   #Durchlaufen aller geparsten Sätze
            Worddict = {"ROOT": [], "Subjekt": [], "AkkusativObjekt": [], "DativObjekt": [], "Rest": []}
            for token in case_doc:   #Zuordnen der Wörter zu den entsprechenden "Wortarten"
                if token.dep_ == "ROOT":
                    Worddict["ROOT"].append(token.text) 
                elif token.dep_ == "sb":
                    Worddict["Subjekt"].append(token.text) 
                elif token.dep_ == "oa":
                    Worddict["AkkusativObjekt"].append(token.text) 
                elif token.dep_ == "da":
                    Worddict["DativObjekt"].append(token.text)
                else:
                    Worddict["Rest"].append(token.text) 
            sentence_dict[x] = (case_doc, Worddict)   #Eintrag mit Satznummer als Key in sentence_dict hinzufügen
            x = x+1
        case_dict[case['id']] = sentence_dict   #Ablegen des sentence_dicts in das case_dict mit Case-ID als Key

        if (counter%(case_range[1] - case_range[0]) == 0 and counter != 0):   #Debugausgabe
            print("counter reached at time: ", str(dt.datetime.now()))
            print(counter)
        counter += 1 
        
    return case_dict

Aufgrund der erheblichen Datei-Größe eines einzigen Case-Dicts für alle Cases ist es erforderlich diese auf mehrere (21) Case-Dicts für jeweils 5000 Cases aufzusplitten. Dies wird im nachfolgenden durchgeführt und und entsprechende Case-Dicts als JSON-Datei nummeriert abgelegt.

Mit Hilfe des Tuples "range_tuple" wird der entsprechende Bereich in "cases" definiert, für welchen im aktuellen Arbeitsschritt ein Case-Dict angelegt werden soll.

In [ ]:
num_dicts = math.ceil(len(cases)/5000)   #Anzahl der nötigen case_dicts berechnen, "math.ceil()" rundet auf

for i in range(num_dicts):
    if(i<(num_dicts-1)):
        range_tuple = (i*5000, (i+1)*5000)
    else:
        range_tuple = (i*5000, len(cases))
    case_dict= get_case_dict(range_tuple)   #Funktion get_case_dict liefert das entsprechende case-dict zurück
    case_dict_df = pd.DataFrame(case_dict)   #Umformen in einen Pandas Dataframe für eine erleichterte Ausgabe als JSON
    export_file_path_json_case_dict = "../case_dict_" +str(i) +"_1000.json"
    case_dict_df.to_json(export_file_path_json_case_dict, force_ascii=False, default_handler=str)
    del case_dict   #Löschen der nicht mehr benötigten Variablen, um Arbteitsspeicher zu schonen
    del case_dict_df   #Löschen der nicht mehr benötigten Variablen, um Arbteitsspeicher zu schonen